## Crawler para extração de dados (Shopper)
Autor: Gustavo F Penedo

##### 1.1 Carregar Requisitos

In [1]:
import pandas as pd
import undetected_chromedriver.v2 as uc
import sys
sys.path.append("..") # Upscale notebooks project folder

from src.crawler.shopper import ShopperCrawler
from src.crawler.config import CrawlerConfig
from src.credentials import Credential
from src.tools import store_data, latin_to_utf8

#### 1.2 Carregar Configurações

In [2]:
# Load Crawler Settings
models = CrawlerConfig().models

# Load Bot Credentials
credential = Credential('shopper')

# Start Crawler Object
shopper_crawler = ShopperCrawler(
    s_key=credential.session_key,
    s_password=credential.session_password)

### 2. Crawling and Parsing

Primeiramente inicia-se o chromedriver para o experimento.

In [3]:
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = shopper_crawler.initialize_driver(uc.Chrome(options=chrome_options))

##### 2.1 Efetuar login e acessar página de extração

Efetua-se o login na página e redireciona para a página de extração

In [4]:
# Login
shopper_crawler.login()

# Página de extração
shopper_crawler.access_component_by_xpath('//button')[0].click()
shopper_crawler.access_url('https://programada.shopper.com.br/shop/alimentos/')

#### 2.2 Carregar cada produto da página de extração

O método irá acessar categoria por categoria e irá carregar todos os produtos na tela, para que em seguida faça a extração dos modals de cada produto.

In [5]:
parsed_products = shopper_crawler.extract_products()

Loading Products: |||||||||||||||||||||||||||||||||||||||||||||||||| 100.0%


### 3. Formatação

Nessa seção será feito a formatação dos dados extraídos para se enquadrar na proposta do problema.

In [6]:
df = pd.DataFrame(parsed_products)
df.price_to = df.price_to.apply(lambda x: float(x.replace(',', '.')) if x else None)

df_assortment = df.copy()
df_sellers = df.copy()
df_sellers['seller_store'] = df_sellers.store
df_sellers['discount_store'] = df_sellers.discount
df_sellers['price_store'] = df_sellers.price_to

seller_players = pd.DataFrame(list(shopper_crawler.sellers.keys()), columns=['seller_player',])
df_sellers = df_sellers.merge(seller_players, how='cross')
df_sellers['price_player'] = df_sellers[['sellers', 'seller_player']].apply(
    lambda x: x['sellers'][x['seller_player']] if x['seller_player'] in x['sellers'] else None,
    axis=1
)
df_sellers = df_sellers[df_sellers.price_player.notnull()]

### 4. Armazenamento

##### 4.1 Armazena assortment.csv

In [7]:
model = 'assortment'
store_data( df_assortment,
            shopper_crawler.models[model]['path'], 
            shopper_crawler.models[model]['columns']
          )

../data/storage/assortment.csv Created


##### 4.2 Armazena sellers.csv

In [8]:
model = 'sellers'
store_data( df_sellers,
            shopper_crawler.models[model]['path'], 
            shopper_crawler.models[model]['columns']
          )


../data/storage/sellers.csv Created
